In [ ]:
# Library for speech processsing

!pip install pydub
!cd /content/drive/MyDrive/04-AudioClassification

In [3]:
# Imports

import os
from pathlib import Path
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import make_chunks

In [ ]:
# Merge multi file of one class

source = "/content/drive/MyDrive/04-AudioClassification"
audios_files = os.path.join(source, "audios_raw")
merged_files_directory = "merged_audios"

Path(os.path.join(source, merged_files_directory)).mkdir(parents=True, exist_ok=True)

for audio in os.listdir(audios_files):
  voice_file1 = os.path.join(audios_files, audio)

  print(voice_file1)
  if "_" in Path(audio).stem:
    if Path(audio).stem.split("_")[1] == '1':
      voice_file2 = voice_file1.replace('1','2')
      voice1 = AudioSegment.from_file(voice_file1)
      voice2 = AudioSegment.from_file(voice_file2)

      combined_voice = voice1 + voice2
      combined_voice.export(os.path.join(source, merged_files_directory, audio.split("_")[0] + ".ogg"), format="ogg", bitrate="192k")
  else:
    voice = AudioSegment.from_file(voice_file1)
    voice.export(os.path.join(source, merged_files_directory, audio), format="ogg", bitrate="192k")
    print(os.path.join(source, merged_files_directory, audio))


In [ ]:
# Preprocessing: Remove silence of audios
"""
split_on_silence: for remove silence of audios
"""

sound = AudioSegment.from_ogg(os.path.join(source, "merged_audios/nahid.ogg"))
audio_chunks = split_on_silence(sound, min_silence_len=2000, silence_thresh=-45)
audio_chunks

In [ ]:
combined_audio = sum(audio_chunks)
combined_audio

In [ ]:
audios_raw_dir_path = os.path.join(source, "merged_audios")
audios_remove_silence_dir_path = os.path.join(source, "audios_remove_silence")
Path(audios_remove_silence_dir_path).mkdir(exist_ok=True)

for file in os.listdir(audios_raw_dir_path):
    file_path = os.path.join(audios_raw_dir_path, file)
    audio = AudioSegment.from_file(file_path)
    
    # draw
    print(file)
    samples = audio.get_array_of_samples()
    plt.figure(figsize=(20, 2), dpi=80)
    plt.plot(samples)
    plt.show()
    
    # remove silence
    audio_chunks = split_on_silence(audio, min_silence_len = 2000, silence_thresh = -45)
    combined_audio = sum(audio_chunks)
    filename = Path(file_path).stem
    output_file_path = os.path.join(audios_remove_silence_dir_path, filename + ".wav")
    combined_audio.export(output_file_path, format="wav", bitrate="32k")

In [ ]:
# Convert audios files to 1sec audios files
"""
make_chunks: convert audios to 1sec files
"""

audios_remove_silence_dir_path = os.path.join(source, "audios_remove_silence")
dataset_dir_path = os.path.join(source,"final_dataset")
Path(dataset_dir_path).mkdir(exist_ok=True)

for file in os.listdir(audios_remove_silence_dir_path):
    filename = Path(file).stem
    file_path = os.path.join(audios_remove_silence_dir_path, file)
    Path(os.path.join(dataset_dir_path, filename)).mkdir(parents=True, exist_ok=True)

    audio = AudioSegment.from_file(file_path) 
    chunk_length_ms = 1000  # pydub calculates in millisec 
    chunks = make_chunks(audio, chunk_length_ms)  #  Make chunks of one sec 
    
    for i, chunk in enumerate(chunks):
        if len(chunk) < 1000:
            continue
        output_file_path = os.path.join(dataset_dir_path, filename, filename + "_" + str(i) + ".wav")
        print ("exporting", output_file_path)
        chunk.export(output_file_path, format="wav")